# Data Workshop 2

**Instructor:** YOUR NAME @YOUREMAIL  
**TAs:** TA NAMES AND EMAILS

[Brief summary of lesson]

Lecture notes adapted from/ inspired by (if applicable)

## 1 | FIRST SECTION

### 1.1 FIRST SUBSECTION